In [155]:
from etl.extract import ProjectZero
from etl.utilities import Utilities
from scipy.stats import zscore
import pandas as pd

design_data = ProjectZero().get_data()['hz_model'].copy()
training_data = Utilities().get_training_data().copy()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
training_data.head(3)

,building_id,plot_id,building_typology,building_gfa,primary_gfa,secondary_typology,secondary_gfa,year_built,occupancy,num_buildings,electricity_demmand
0,4593574,10019,office,169416,164754.0,bank_branch,4662.0,1909,95,1,1920103.6
1,2967701,11373,education,94380,94380.0,education,94380.0,1963,100,1,180640.0
3,2917939,10001,hotel,50000,50000.0,hotel,50000.0,1994,100,1,579335.2


In [157]:
design_data.head()

,ID,Typology,Area,Volume,Plot,Building,Envelope,Elevation,Colour
0,37108380-bb59-4614-9e39-2de7f9036322,Retail,400,1400,3,Tower 03,280.0,0,#FF5442
1,a3b644fb-21ee-468c-b31f-fae4b5662842,Retail,400,1400,3,Podium 03,280.0,0,#FF5442
2,0ea705aa-80aa-4619-9d9a-b86fd3f0c322,Retail,400,1400,2,Tower 02,280.0,0,#FF5442
3,36d9da83-6389-45b1-8b66-5300aae9d977,Retail,400,1400,1,Tower 01,280.0,0,#FF5442
4,56a2686a-5346-4f86-8b84-d9f7de354eaf,Community,400,1400,1,Podium 01,280.0,0,#96D200


In [158]:
# drop irrelevant columns
design_data.drop(columns=['ID','Volume','Elevation','Colour', 'Envelope'], inplace=True)

In [159]:
design_data.Building.unique()

array(['Tower 03', 'Podium 03', 'Tower 02', 'Tower 01', 'Podium 01',
       'Podium 02'], dtype=object)

In [160]:
# Rank areas within each building type
design_data['Area_Rank'] = design_data.groupby('Building')['Area'].rank(ascending=False)

# Sort by building and area rank
design_data = design_data.sort_values(by=['Building', 'Area_Rank'])

# Rank areas within each building and typology
design_data['Area_Rank'] = design_data.groupby(['Building', 'Typology'])['Area'].rank(ascending=False)

# Sort by building, typology, and area rank
design_data = design_data.sort_values(by=['Building', 'Typology', 'Area_Rank'])

# Set index before groupby to preserve all rows
design_data.set_index(['Plot', 'Building', 'Typology', 'Area_Rank'], inplace=True)

# Group by index and calculate the sum
result_df = design_data.groupby(level=[0, 1, 2, 3]).sum()

# Reset index to bring back the original DataFrame structure
result_df.reset_index(inplace=True)
result_df

,Plot,Building,Typology,Area_Rank,Area
0,1,Podium 01,Community,1.0,400
1,1,Tower 01,Office,3.0,2000
2,1,Tower 01,Retail,1.0,400
3,2,Podium 02,Education,1.0,400
4,2,Tower 02,Residential,2.0,1200
5,2,Tower 02,Retail,1.0,400
6,3,Podium 03,Retail,1.0,400
7,3,Tower 03,Hotel,1.5,800
8,3,Tower 03,Retail,1.0,400


In [161]:
# Find the index of the maximum 'Area_Rank' within each group
max_rank_index = result_df.groupby(['Plot', 'Building'])['Area_Rank'].idxmax()
max_rank_index

# Create a new DataFrame with the highest ranked Typology and Area for each building
primary_asset_df = result_df.loc[max_rank_index, ['Plot', 'Building', 'Typology', 'Area_Rank', 'Area']].reset_index(drop=True)
primary_asset_df

# Rename the columns for clarity
primary_asset_df.rename(columns={'Typology': 'Primary_Asset', 'Area': 'Primary_Asset_Area'}, inplace=True)
primary_asset_df

# Merge the primary_asset_df back to the original DataFrame without automatic renaming of 'Area_Rank'
result_df = pd.merge(result_df, primary_asset_df, left_on=['Plot', 'Building', 'Area_Rank'], right_on=['Plot', 'Building', 'Area_Rank'], how='left')
result_df

# Exclude the rows corresponding to the maximum rank
result_df_excluded_max = result_df[~result_df.index.isin(max_rank_index)]

# Find the index of the second maximum 'Area_Rank' within each group in the remaining rows
second_max_rank_index = result_df_excluded_max.groupby(['Plot', 'Building'])['Area_Rank'].idxmax()

# Create a new DataFrame with the second highest ranked Typology and Area for each building
second_primary_asset_df = result_df_excluded_max.loc[second_max_rank_index, ['Plot', 'Building', 'Typology', 'Area_Rank', 'Area']].reset_index(drop=True)

# Rename the columns for clarity
second_primary_asset_df.rename(columns={'Typology': 'Second_Asset', 'Area': 'Second_Asset_Area'}, inplace=True)

# Merge the second_primary_asset_df back to the original DataFrame without automatic renaming of 'Area_Rank'
result_df = pd.merge(result_df, second_primary_asset_df, left_on=['Plot', 'Building', 'Area_Rank'], right_on=['Plot', 'Building', 'Area_Rank'], how='left')
result_df


,Plot,Building,Typology,Area_Rank,Area,Primary_Asset,Primary_Asset_Area,Second_Asset,Second_Asset_Area
0,1,Podium 01,Community,1.0,400,Community,400.0,NaN,NaN
1,1,Tower 01,Office,3.0,2000,Office,2000.0,NaN,NaN
2,1,Tower 01,Retail,1.0,400,NaN,NaN,Retail,400.0
3,2,Podium 02,Education,1.0,400,Education,400.0,NaN,NaN
4,2,Tower 02,Residential,2.0,1200,Residential,1200.0,NaN,NaN
5,2,Tower 02,Retail,1.0,400,NaN,NaN,Retail,400.0
6,3,Podium 03,Retail,1.0,400,Retail,400.0,NaN,NaN
7,3,Tower 03,Hotel,1.5,800,Hotel,800.0,NaN,NaN
8,3,Tower 03,Retail,1.0,400,NaN,NaN,Retail,400.0


In [162]:
agg_result = result_df.groupby(['Building','Plot']).agg({
    'Area': 'sum',
    'Primary_Asset': 'first',  # Assuming 'Primary_Asset' is the same for all rows within a building
    'Primary_Asset_Area': 'sum',
    'Second_Asset': 'first',   # Assuming 'Second_Asset' is the same for all rows within a building
    'Second_Asset_Area': 'sum'  # Assuming 'Second_Asset_Area' is the same for all rows within a building
}).reset_index()

agg_result

,Building,Plot,Area,Primary_Asset,Primary_Asset_Area,Second_Asset,Second_Asset_Area
0,Podium 01,1,400,Community,400.0,None,0.0
1,Podium 02,2,400,Education,400.0,None,0.0
2,Podium 03,3,400,Retail,400.0,None,0.0
3,Tower 01,1,2400,Office,2000.0,Retail,400.0
4,Tower 02,2,1600,Residential,1200.0,Retail,400.0
5,Tower 03,3,1200,Hotel,800.0,Retail,400.0


In [163]:
#Comparing against training set
training_data.head(3)

,building_id,plot_id,building_typology,building_gfa,primary_gfa,secondary_typology,secondary_gfa,year_built,occupancy,num_buildings,electricity_demmand
0,4593574,10019,office,169416,164754.0,bank_branch,4662.0,1909,95,1,1920103.6
1,2967701,11373,education,94380,94380.0,education,94380.0,1963,100,1,180640.0
3,2917939,10001,hotel,50000,50000.0,hotel,50000.0,1994,100,1,579335.2


In [164]:
from datetime import datetime
# Get the current date
current_year = datetime.now().year

# adding missing columns
agg_result['year_built'] = current_year
agg_result['occupancy'] = 100
agg_result['num_buildings'] = 1

agg_result


,Building,Plot,Area,Primary_Asset,Primary_Asset_Area,Second_Asset,Second_Asset_Area,year_built,occupancy,num_buildings
0,Podium 01,1,400,Community,400.0,None,0.0,2024,100,1
1,Podium 02,2,400,Education,400.0,None,0.0,2024,100,1
2,Podium 03,3,400,Retail,400.0,None,0.0,2024,100,1
3,Tower 01,1,2400,Office,2000.0,Retail,400.0,2024,100,1
4,Tower 02,2,1600,Residential,1200.0,Retail,400.0,2024,100,1
5,Tower 03,3,1200,Hotel,800.0,Retail,400.0,2024,100,1


In [165]:
# rename and reorder to match training set
preproc_design_data = pd.DataFrame({
    'building_id': agg_result['Building'],
    'plot_id': agg_result['Plot'],
    'building_typology': agg_result['Primary_Asset'].str.lower(),
    'building_gfa': agg_result['Area'],
    'primary_gfa': agg_result['Primary_Asset_Area'],
    'secondary_typology': agg_result['Second_Asset'],
    'secondary_gfa': agg_result['Second_Asset_Area'],
    'year_built': agg_result['year_built'],
    'occupancy': agg_result['occupancy'],
    'num_buildings': agg_result['num_buildings']
    })

preproc_design_data


,building_id,plot_id,building_typology,building_gfa,primary_gfa,secondary_typology,secondary_gfa,year_built,occupancy,num_buildings
0,Podium 01,1,community,400,400.0,None,0.0,2024,100,1
1,Podium 02,2,education,400,400.0,None,0.0,2024,100,1
2,Podium 03,3,retail,400,400.0,None,0.0,2024,100,1
3,Tower 01,1,office,2400,2000.0,Retail,400.0,2024,100,1
4,Tower 02,2,residential,1600,1200.0,Retail,400.0,2024,100,1
5,Tower 03,3,hotel,1200,800.0,Retail,400.0,2024,100,1


In [166]:
# Define feature engineering functions
def calculate_occupied_area(data):
    data["occupied_area"] = data['building_gfa'] * (data['occupancy'] * 0.01)
    return data

def calculate_new_build(data):
    data["new_build"] = 0  # Initialize to 0
    
    # Update to 1 for rows where the condition is met
    data.loc[data['year_built'] >= 2000, 'new_build'] = 1
    
    return data

In [167]:
# test
# load pickle back into test
import pickle
pipe = pickle.load(open('pipeline/pipeline.pkl', 'rb'))

In [168]:
preproc_design_data.reset_index(drop=True, inplace=True)
preproc_design_data['electricity_demmand'] = pipe.predict(preproc_design_data).astype(int)
preproc_design_data

,building_id,plot_id,building_typology,building_gfa,primary_gfa,secondary_typology,secondary_gfa,year_built,occupancy,num_buildings,electricity_demmand
0,Podium 01,1,community,400,400.0,None,0.0,2024,100,1,259207
1,Podium 02,2,education,400,400.0,None,0.0,2024,100,1,259207
2,Podium 03,3,retail,400,400.0,None,0.0,2024,100,1,323657
3,Tower 01,1,office,2400,2000.0,Retail,400.0,2024,100,1,353009
4,Tower 02,2,residential,1600,1200.0,Retail,400.0,2024,100,1,101781
5,Tower 03,3,hotel,1200,800.0,Retail,400.0,2024,100,1,233543


In [169]:
#Comparing against training set
training_data.head(3)

,building_id,plot_id,building_typology,building_gfa,primary_gfa,secondary_typology,secondary_gfa,year_built,occupancy,num_buildings,electricity_demmand
0,4593574,10019,office,169416,164754.0,bank_branch,4662.0,1909,95,1,1920103.6
1,2967701,11373,education,94380,94380.0,education,94380.0,1963,100,1,180640.0
3,2917939,10001,hotel,50000,50000.0,hotel,50000.0,1994,100,1,579335.2
